In [8]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element, dump, ElementTree
import random
from tqdm import tqdm
import math

In [9]:
def adress(root_path,file):
    file_list = os.listdir(root_path)
    file_list = [file_name for file_name in file_list if file_name.endswith(file)]
    return root_path,file_list

In [10]:
def CROP(num):
    
    if(num==0):
        return
    
    class CropLayer(object):
        def __init__(self, params, blobs):
            self.xstart = 0
            self.xend = 0
            self.ystart = 0
            self.yend = 0

        def getMemoryShapes(self, inputs):
            inputShape, targetShape = inputs[0], inputs[1]
            batchSize, numChannels = inputShape[0], inputShape[1]
            height, width = targetShape[2], targetShape[3]

            self.ystart = (inputShape[2] - targetShape[2]) // 2
            self.xstart = (inputShape[3] - targetShape[3]) // 2
            self.yend = self.ystart + height
            self.xend = self.xstart + width

            return [[batchSize, numChannels, height, width]]

        def forward(self, inputs):
            return [inputs[0][:,:,self.ystart:self.yend,self.xstart:self.xend]]


    def GuidedFilt(img, r):
        '''
           https://github.com/Utkarsh-Deshmukh/Edge-Preserving-smoothing-Filter-Comparison

        '''
        eps = 0.04;

        I = np.double(img);
        I = I/255;
        I2 = cv2.pow(I,2);
        mean_I = cv2.boxFilter(I,-1,((2*r)+1,(2*r)+1))
        mean_I2 = cv2.boxFilter(I2,-1,((2*r)+1,(2*r)+1))

        cov_I = mean_I2 - cv2.pow(mean_I,2);

        var_I = cov_I;

        a = cv2.divide(cov_I,var_I+eps)
        b = mean_I - (a*mean_I)

        mean_a = cv2.boxFilter(a,-1,((2*r)+1,(2*r)+1))
        mean_b = cv2.boxFilter(b,-1,((2*r)+1,(2*r)+1))

        q = (mean_a * I) + mean_b;

        return(np.uint8(q*255))

    def movingAverageFilter(signal, N):
        n = N - 1
        if n % 2 == 0:
            idx1 = int(n / 2)
            idx2 = idx1
        else:
            n2 = n - 1
            idx1 = int(n2 / 2)
            idx2 = idx1 + 1

        return np.convolve(signal, np.ones((N,))/N)[idx1 : -idx2]

    def getCorner(img):
        cnts, _ = cv2.findContours(img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        idx = np.argsort([cv2.contourArea(cnt) for cnt in cnts])[::-1][0] #
        cnt = np.array(cnts)[idx]
        info = cv2.minAreaRect(cnt)
        rotatedBbox = cv2.boxPoints(info).astype(np.int)

        sm = rotatedBbox.sum(axis = 1)
        diff =  np.diff(rotatedBbox, axis = 1)
        lt = rotatedBbox[np.argmin(sm)]
        rb = rotatedBbox[np.argmax(sm)]
        rt = rotatedBbox[np.argmin(diff)]
        lb = rotatedBbox[np.argmax(diff)]

        points = [lt, rt, rb, lb]
        return points

    def getDegree(pt1, pt2):
        dist1 = np.sqrt((pt1[0] - pt2[0])**2 + (pt1[1] - pt2[1])**2) + 1e-10
        dist2 = abs(pt1[0] - pt2[0])
        degree = np.rad2deg(np.arccos(dist2 / dist1))
        return degree

    def adjustPoint(points, adjust_x):
        adjust_y = int(adjust_x * 1.5)
        points = np.array(points)
        sm = points.sum(axis = 1)
        diff =  np.diff(points, axis = 1)
        lt = points[np.argmin(sm)] + [adjust_x, adjust_y]
        rb = points[np.argmax(sm)] + [-adjust_x, -adjust_y]
        rt = points[np.argmin(diff)] + [-adjust_x, adjust_y]
        lb = points[np.argmax(diff)] + [adjust_x, -adjust_y]
        return np.array([lt, rt, rb, lb])


    def findMax(maf, width, thr):
        diff = np.diff(maf)
        idx, val = [], []

        for n in range(len(diff)): # 어쩌면 여기서 한 개 차이로 에러날 수도 있음
            if n < width:
                if diff[n] >= 0 and diff[n + 1] < 0:
                    max_ = maf[n]
                    v1 = maf[0]
                    v2 = maf[n + width]
                    v3 = np.minimum(v1, v2)        

                    if v1 < max_ and max_ > v2 and (max_ - v3) >= thr:
                        idx.append(n)
                        val.append(maf[n])

            elif n >= (len(diff) - width):
                if diff[n] >= 0 and diff[n + 1] < 0:
                    max_ = maf[n]
                    v1 = maf[n - width]
                    v2 = maf[-1]
                    v3 = np.minimum(v1, v2)        

                    if v1 < max_ and max_ > v2 and (max_ - v3) >= thr:
                        idx.append(n)
                        val.append(maf[n])

            else:
                if diff[n] >= 0 and diff[n + 1] < 0:
                    max_ = maf[n]
                    v1 = maf[n - width]
                    v2 = maf[n + width]
                    v3 = np.minimum(v1, v2)

                    if v1 < max_ and max_ > v2 and (max_ - v3) >= thr:
                        idx.append(n)
                        val.append(maf[n])

        return idx, val

    def findMax2(ori, maf, width, thr):
        diff = np.diff(maf)
        idx, val = [], []

        for n in range(len(diff)):
            if n < width:
                if diff[n] >= 0 and diff[n + 1] < 0:
                    max_ = ori[n]
                    v1 = ori[0]
                    v2 = ori[n + width]
                    v3 = np.minimum(v1, v2)        

                    if v1 < max_ and max_ > v2 and (max_ - v3) >= thr:
                        idx.append(n)
                        val.append(maf[n])

            elif n >= (len(diff) - width):
                if diff[n] >= 0 and diff[n + 1] < 0:
                    max_ = ori[n]
                    v1 = ori[n - width]
                    v2 = ori[-1]
                    v3 = np.minimum(v1, v2)        

                    if v1 < max_ and max_ > v2 and (max_ - v3) >= thr:
                        idx.append(n)
                        val.append(maf[n])

            else:
                if diff[n] >= 0 and diff[n + 1] < 0:
                    max_ = ori[n]
                    v1 = ori[n - width]
                    v2 = ori[n + width]
                    v3 = np.minimum(v1, v2)        

                    if v1 < max_ and max_ > v2 and (max_ - v3) >= thr:
                        idx.append(n)
                        val.append(maf[n])

        return idx, val

    def detectEdge(img, net, alpha):
        # HED
        h, w, c = img.shape
        blob = cv2.dnn.blobFromImage(img, scalefactor = 1.0, size = (w, h),
                                     mean = (104.00698793, 116.66876762, 122.67891434),
                                     swapRB = False, crop = False)
        net.setInput(blob)
        hed = net.forward()
        hed = cv2.resize(hed[0,0], (w, h))
        hed = (255 * hed).astype(np.uint8)
        hed = GuidedFilt(hed, alpha)

        return hed

    def ppImg(img, adjust1 = 30, adjust2 = 5):
        '''
          step 1 - 최외곽 윤곽선에서 안으로 일정 pixel씩 들여넣어 사각형 그림

                   일부 이미지의 경우 세세한 영역의 edge까지 detection 되어서 이를 제거하기 위함
        '''
        corner = getCorner(img)
        corner = adjustPoint(corner, adjust1)
        tmp = (255 - img).astype(np.uint8)
        res = cv2.drawContours(tmp, [corner], 0, (255), -1)


        '''
          step 2 - 일부 이미지에서는 외곽선이 끊어지는 경우가 존재해 이를 보완하고자 적용

        '''
        cnts, _ = cv2.findContours(res, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        idx = np.argsort([cv2.contourArea(cnt) for cnt in cnts])[::-1][1] #
        cnt = np.array(cnts)[idx]
        info = cv2.minAreaRect(cnt)
        rotatedBbox = cv2.boxPoints(info).astype(np.int)
        rotatedBbox = adjustPoint(rotatedBbox, adjust2)
        res = cv2.drawContours(res, [rotatedBbox], 0, (0), 5)

        return res

    def cropKit(edge, img, width, height):
        H, W, C = img.shape # original shape의 image 받아야함
        h, w = edge.shape

        cnts, _ = cv2.findContours(edge, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        idx = np.argsort([cv2.contourArea(cnt) for cnt in cnts])[::-1][2] #
        cnt = np.array(cnts)[idx]
        info = cv2.minAreaRect(cnt)
        rotatedBbox = cv2.boxPoints(info).astype(np.int)

        sm = rotatedBbox.sum(axis = 1)
        diff =  np.diff(rotatedBbox, axis = 1)
        lt = rotatedBbox[np.argmin(sm)] * [W / w, H / h]
        rb = rotatedBbox[np.argmax(sm)] * [W / w, H / h]
        rt = rotatedBbox[np.argmin(diff)] * [W / w, H / h]
        lb = rotatedBbox[np.argmax(diff)] * [W / w, H / h]

        p1, p2 = (0, 0), (0, height)
        p3, p4 = (width, 0), (width, height)
        M = cv2.getPerspectiveTransform(np.float32([lt, lb, rt, rb]), np.float32([p1, p2, p3, p4]))
        res = cv2.warpPerspective(img, M, (width, height))

        return res

    def cropReactionZone(img):
        h, w, c = img.shape
        resized = cv2.resize(img, (80,400))    
        signal = np.mean(255 - resized, axis = 1)[...,2]
        maf = movingAverageFilter(signal, 15)
        x, y = findMax(maf, 30, 0)

        start = 0
        for i in range(len(x) - 1):
            if x[i + 1] - x[i] > 30:
                center_x = int((x[i + 1] + x[i]) / 2)
                center_y = signal[center_x]
                for j in range(x[i], center_x - 1, 5):                    
                    degree = getDegree((center_x, center_y), (j, signal[j]))
                    if degree < 10:
                        start = j           
                        break
                start = start if start != 0 else x[i]
                break   

        end = 0
        for i in range(1, len(x) - 2):
            if x[-i] - x[-i - 1] > 30:
                center_x = int((x[-i - 1] + x[-i]) / 2)
                center_y = signal[center_x]
                for j in range(center_x + 1, x[-i], 1)[::-1]:
                    degree = getDegree((center_x, center_y), (j, signal[j]))
                    if degree < 10:
                        end = j
                        break
                end = end if end != 0 else x[-i]
                break

        Xstart = int(start / 400 * h)
        Xend = int(end / 400 * h)

        return img[Xstart : Xend, :]

    def analyzeKit(img, width = 30, thr = 1):
        chA = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)[..., 1]
        signal = np.mean(chA, axis = 1)
        maf = movingAverageFilter(signal, 15)
        x, y = findMax2(signal, maf, width, thr)
        x = list(np.array(x) - 8)

        return x, y, maf[8:-8]

    def CoviCareAI(img, net, alpha):
        #img = cv2.resize(img, (400, 600))
        original = img.copy()
        img = cv2.resize(img, (400, 600))

        # edge detection / HED
        edge = detectEdge(img, net, alpha)
        _, thr = cv2.threshold(edge, int(np.max(edge) * 2 / 5), 255, cv2.THRESH_BINARY)

        # preprocess image
        ppimg = ppImg(thr)

        # crop kit
        kit = cropKit(ppimg, original, 1200, 600)
        _,kit=cv2.threshold(kit,127,255,cv2.THRESH_TOZERO)
        point1=[]
        point2=[]
        for x in range(200):
            if(all(kit[0,x])==0):
                point1=[0,x]
            if(all(kit[599,x])==0):
                point2=[599,x]
#600 300
        angle=0
        if (int(point1[1])>int(point2[1])):
            angle=math.atan((point1[1]-point2[1])/599)
            M1=cv2.getRotationMatrix2D((600,300),angle*180.0/math.pi,1)
            kit=cv2.warpAffine(kit,M1,(1200,600))
        elif (int(point1[1])<int(point2[1])):
                angle=math.atan((point2[1]-point1[1])/599)
                M2=cv2.getRotationMatrix2D((600,300),360.0-(angle*180.0/math.pi),1)
                kit=cv2.warpAffine(kit,M2,(1200,600))

        return kit
    
    root_path,TYPE_list=adress('/Data/HT/PAX/PAX_file/original/TYPE 1/','.jpg')
    cv2.dnn_registerLayer('Crop', CropLayer)
    net = cv2.dnn.readNetFromCaffe('/Data/HT/PAX/PAX_file/deploy.prototxt', '/Data/HT/PAX/PAX_file/hed_pretrained_bsds.caffemodel')
    
    for img_name in TYPE_list:
        img = cv2.imread(root_path + img_name)
        results = CoviCareAI(img, net, alpha = 23)
        #results = cv2.cvtColor(results, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(15,15))
        cv2.imwrite("/Data/HT/PAX/PAX_file/crop_img/"+img_name,results)
    return

In [4]:
def POINT_LABELING(num):
    i=0
    if(num=='0'):
        return
    root_path1, original_list=adress('/Data/HT/PAX/PAX_file/crop_img/','.jpg')
    root_path2,labeling_list=adress('/Data/HT/PAX/PAX_file/labeling/','.xml')
    kernel=np.ones((5,5),np.float32)/25
    
    for img_name,labeling_name in zip(original_list,labeling_list):
        img = cv2.imread(root_path1+img_name)
        
        doc=ET.parse(root_path2+labeling_name)
        
        root=doc.getroot()
        
        for bndbox in root.iter("bndbox"):
            xmin=bndbox.findtext("xmin")
            ymin=bndbox.findtext("ymin")
            xmax=bndbox.findtext("xmax")
            ymax=bndbox.findtext("ymax")
            
            point=[int(xmin),int(ymin),int(xmax),int(ymax)]
            point_img=img[point[1]:point[3],point[0]:point[2]]
            #point_img= cv2.cvtColor(point_img, cv2.COLOR_BGR2RGB)
            
            cv2.imwrite("/Data/HT/PAX/PAX_file/point_img/"+str(i)+".jpg",point_img)
            i=i+1
            img[point[1]-2:point[3]+2,point[0]:point[2]]=img[point[1]-2-(point[3]-point[1]+4):point[1]-2,point[0]:point[2]]
            dst=cv2.filter2D(img[point[1]-2:point[3]+2,point[0]:point[2]],-1,kernel)
            
        cv2.imwrite("/Data/HT/PAX/PAX_file/elimination_img/"+img_name,img)
    return

In [5]:
def DATA(nData):
    root_path1,point_test_beta_list=adress("/Data/HT/PAX/PAX_file/point_img/",'jpg')
    root_path2,IMG_list=adress("/Data/HT/PAX/PAX_file/elimination_img/",'.jpg')
    
    x_list=[35,195,352,508,664,820,973,1128]
    y_list=[230,258,285,313,342]
    
    for data in range(int(nData)):
        img=cv2.imread(root_path2+IMG_list[random.randint(0,len(IMG_list)-1)])
        #이미지가 하나말곤 없어서 사용 못함
        #img=cv2.imread("C:/Users/JLK/Desktop/elimination_img/20201026_100920.jpg")
        _,mask_img=cv2.threshold(img,255,255,cv2.THRESH_BINARY)
        for y in y_list:
            for x in x_list:
                if(random.randint(1,2)%2==0):
                    labeling_img=cv2.imread(root_path1+point_test_beta_list[random.randint(0,len(point_test_beta_list)-1)])
                    _,mask_labeling_img=cv2.threshold(labeling_img,0,255,cv2.THRESH_BINARY)
                    heigh,width,_=labeling_img.shape
                    img[y:y+heigh,x:x+width]=0
                    img[y:y+heigh,x:x+width]=labeling_img
                    mask_img[y:y+heigh,x:x+width]=mask_labeling_img
        cv2.imwrite("/Data/HT/PAX/201111_train_data/final/"+str(data)+'.jpg',img)
        cv2.imwrite("/Data/HT/PAX/201111_train_data/final_mask/"+str(data)+"_mask.jpg",mask_img)
    return

In [ ]:
#주소확인
nData=input()#실행횟수
#num=input()#0=off 1=on
#CROP(num)
#POINT_LABELING(num)
DATA(nData)

100000
